In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_APIKEY")
print("Api is loaded") if api_key else print("Api is not available")

Api is loaded


#### Openai agent sdk with groq key

In [2]:
# required if in case of run in jupyter notebook
import nest_asyncio
nest_asyncio.apply()


from agents import Agent, Runner, function_tool, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
groq_api_key = api_key #os.getenv('GROQ_API_KEY')
GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)


agent1 = Agent(name="Lead Calling Agent English", instructions="You are a helpful calling Agent. Your will start the conversation You will say I am calling from Saginfotech. Are you interested in eazemax and gencodex software tool. and write back response of user query if you have any prier knowledge ",model=llama3_3_model)

agent2 = Agent(name="Lead Calling Agent Hindi", instructions="You are a helpful calling Agent. Your will start the conversation  in Hindi You will say 'Main Saginfotech se baat kar raha hu . Ky app Eazemax Aur Gencodex software main interested hai. ", model = llama3_3_model )

result1 = Runner.run_sync(agent1,"start the conversation, what you can do for me?")
result2 = Runner.run_sync(agent2,"start the conversation")
print("Agent 1 response : ",result1.final_output)
print("Agent 2 response : ",result2.final_output)


Agent 1 response :  I am calling from Saginfotech. Are you interested in Eazemax and Gencodex software tools? We specialize in providing innovative software solutions to streamline business operations and improve efficiency. Our Eazemax and Gencodex tools are designed to help organizations like yours automate tasks, enhance productivity, and make data-driven decisions. Would you like to know more about how our software can benefit your business?
Agent 2 response :  Main Saginfotech se baat kar raha hu . Ky app Eazemax Aur Gencodex software main interested hai?


OPENAI_API_KEY is not set, skipping trace export


#### Groq api calling

In [3]:
from groq import Groq

client = Groq(api_key=api_key)

chat_completion = client.chat.completions.create(
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        # Set a user message for the assistant to respond to.
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],

    # The language model which will generate the completion.
    model="llama-3.3-70b-versatile"
)

# Print the completion returned by the LLM.
print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technological landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models can process and analyze vast amounts of text data quickly, which is essential for applications that require real-time or near-real-time responses, such as chatbots, virtual assistants, and language translation software. This speed enables these models to handle a high volume of requests without significant delays.

2. **Resource Optimization**: By being fast, these models can operate on less powerful hardware or use less computational resources (like GPU power) to achieve the same task as slower models. This optimization is vital for deploying language models on edge devices (like smartphones or smart home devices) where computational resources are limited. It also helps in reducing the energy consumption and environmental impact of data centers.

3. **Real-time Interaction**: The ability to respond qui

#### openAi with groq api

In [7]:
import os
import openai


url = "https://api.groq.com/openai/v1"

# ✅ Define agent-like behavior using Groq's LLaMA 3 model
client = openai.OpenAI(base_url=url,api_key=api_key)
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "who is the president and vice precident of india?"}
    ]
)
print(response.choices[0].message.content)


As of my knowledge cutoff, the President of India is:

**Draupadi Murmu**

She is the 15th President of India and took office on July 25, 2022.

And the Vice President of India is:

**Jagdeep Dhankhar**

He is the 14th Vice President of India and took office on August 11, 2022.

Please note that these positions are subject to change, and if you need more up-to-date information, I can try to help you with that!


#### OpenAi Agent SDK Development (handoffs primitive define in agent)

In [20]:
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai import AsyncOpenAI
import asyncio

GROQ_BASE_URL = "https://api.groq.com/openai/v1"
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=api_key)
open_ai_model = "openai/gpt-oss-20b"
llama3_3_model = OpenAIChatCompletionsModel(model=open_ai_model, openai_client=groq_client)


history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model = llama3_3_model
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples", 
    model = llama3_3_model
)



triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent], 
    model = llama3_3_model)



query = input("Ask your query .................    ")
print("question : ",query)

async def main():
    result = await Runner.run(triage_agent, query)
    print("answer : ",result.final_output)


if __name__ == "__main__":
    asyncio.run(main())





question :  who was the prime minister of india in 2015?
answer :  In 2015, the Prime Minister of India was **Narendra Modi**. He assumed office on 26 May 2014 after the Bharatiya Janata Party (BJP) won a decisive victory in the general elections. Modi’s first term (2014‑2019) was marked by a focus on economic reforms, digital initiatives, and efforts to boost foreign investment, setting the stage for the continued development agenda that he carried on in subsequent terms.


#### OpenAi Agent SDK Development Add a guardrail with handoff

In [45]:
import asyncio
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner, InputGuardrailTripwireTriggered

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="You are asked to check if this is query is related to (Math,History) query. Answer clearly in text: 'Yes – reason...' or 'No – reason...'",
    model=llama3_3_model
)

math_tutor_agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llama3_3_model
)

history_tutor_agent = Agent(
    name="History Tutor",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llama3_3_model
)


send_deny_response_agent = Agent(
    name="Deny replier",
    instructions="You are responsible to the write deny response message of query if not related to 'Math and History', reply polietly, and also write reason why you can not help with that query. ",
    model=llama3_3_model)



async def guardrail_function(ctx, agent, user_input):
    result = await Runner.run(guardrail_agent, user_input, context=ctx.context)
    raw = result.final_output.strip().lower()
    # print(raw)
    is_valid_query = raw.startswith("yes")
    reasoning = result.final_output.strip()
    return GuardrailFunctionOutput(
        output_info={"is_homework": is_valid_query, "reasoning": reasoning},
        tripwire_triggered=not is_valid_query
    )

triage_agent = Agent(
    name="Triage Agent",
    instructions="Route homework questions to the right tutor, get response frome the expert tutor.",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[InputGuardrail(guardrail_function=guardrail_function)],
    model=llama3_3_model
)

async def main():
    for q in ["who was the first president of the united states?", "what is the square of 2?","Name of bollywood 3 khan superstar?"]:
        try:
            res = await Runner.run(triage_agent, q)
            print(f"🤖 Response: {res.final_output}")
        except InputGuardrailTripwireTriggered:
            # Use the guardrail result's reasoning to craft a reply
            res = await Runner.run(send_deny_response_agent,f"User is asked this question : {q}") #  <---- 
            print(f"🤖 Deny Response  : {res.final_output}")

if __name__ == "__main__":
    asyncio.run(main())


🤖 Response: The first president of the United States was **George Washington**. He was inaugurated on April 30, 1789, and served two terms until 1797.

### Why he was chosen
- **Reputation and leadership:** Washington had led the Continental Army to victory in the Revolutionary War and was widely respected for his integrity and military skill.
- **Neutrality:** As a relatively young nation, the United States needed a president who could appeal to a broad range of factions and maintain political stability.
- **Constitutional guidance:** The newly drafted Constitution set up a system of checks and balances; Washington’s leadership helped define the executive’s role within that framework.

### Key actions during his presidency
- **Establishing precedents:** Washington set many of the protocols still used today—such as appointing a cabinet, issuing the first executive orders, and laying out a neutral foreign policy stance.
- **Neutrality Proclamation (1793):** Declared the U.S. would not t

#### openAi Agent with Tool configuration.

In [58]:
import asyncio
import os
import requests
from openai import AsyncOpenAI

from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled

set_tracing_disabled(disabled=True)

API_KEY = "25f0559351f3c75faa45800ee42df5b1"  # 🔑 Replace with your key
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"  # login signup and create api


@function_tool
def get_weather(city: str) -> dict:
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric"  # Use "imperial" for Fahrenheit
    }
    
    try:
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        # print(data)
        if response.status_code == 200:
            # print({
            #     "city": data["name"],
            #     "temperature": data["main"]["temp"],
            #     "humidity": data["main"]["humidity"],
            #     "description": data["weather"][0]["description"].capitalize()
            # })
            return {
                "city": data["name"],
                "temperature": data["main"]["temp"],
                "humidity": data["main"]["humidity"],
                "description": data["weather"][0]["description"].capitalize()
            }
        else:
            return {"error": data.get("message", "Unable to fetch weather data")}
    except Exception as e:
        return {"error": str(e)}




async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="Assistant",
        instructions="You are responsive to provide the whether information to a specific, using tool.",
        model=  llama3_3_model,  
        tools=[get_weather],
    )

    result = await Runner.run(agent, "What's the weather in mummai?")
    print(result.final_output)


if __name__ == "__main__":
    asyncio.run(main())

Here’s the current weather in **Mumbai**:

- **Temperature:** 27.5 °C (≈81 °F)  
- **Humidity:** 80 %  
- **Conditions:** Overcast clouds  

Feel free to let me know if you’d like a forecast for the next few days or weather for another city!
